In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime
from selenium import webdriver
import time
import requests
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

#그래프의 축 등에서 음수를 표시할 때 minus sign이 깨지는 것 해결
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

C:\Users\user\AppData\Local\Temp\ipykernel_15140\2025618107.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
default_URL = "https://www.wanted.co.kr/salary/"
wd = webdriver.Chrome()
occs = {'개발' : '518', '경영·비즈니스': '507', '마케팅·광고': '523', '디자인': '511', '영업': '530', '고객서비스·리테일': '510',
        '미디어':'524', 'HR':'517', '금융':'508', '제조·생산':'522', '교육': '10101',
        '의료·제약·바이오':'515'} #주소 뒤의 숫자값
totaldf = pd.DataFrame(columns = ['직군', '직무', '신입', '1년차', '2년차', '3년차' ,'4년차', '5년차', '6년차', '7년차', '8년차', '9년차', '10년차'])
occname = list(occs.keys())
dfnum = 0 #데이터프레임 인덱스값

for h in range(len(occs)): # 직군만큼 반복
    URL = default_URL + occs[occname[h]]
    wd.get(URL)
    time.sleep(2) #과부화 방지
    html = wd.page_source
    wanted = BeautifulSoup(html, 'html.parser')
    jobshtml = wanted.select('div > div > select > option') #직군, 직업, 경력 모두 나옴
    jobs = jobshtml[20:-12] #직업만 뽑아내고 싶은데, 직군이랑 경력의 갯수는 바뀌지 않으므로 직군 20개와 경력 12개 제거
    jobvalue = {} #딕셔너리 초기화
    for g in range(len(jobs)): #직업 갯수만큼 반복
        jobname = jobs[g].string #직업 이름 추출
        a = str(jobs[g]) #value값을 뽑으려 split하기 위해 option~구문을 문자열화
        b = a.split('value="', 1)[1] #value값의 앞까지 한번만 분할, 뒤의내용 추출
        value = b.split('">',1)[0] #value값 이후부터 한번만 분할, 앞의내용 추출
        jobvalue[jobname] = value #직업 이름 key값에 주소 뒤의 숫자인 value값 저장
    jobnames = list(jobvalue.keys()) #key만 따로 리스트로 분할
  
    for k in range(len(jobvalue)): #직업만큼 반복
        if jobvalue[jobnames[k]] == '0': #직업이 value값 0인 '전체' 일시에
            URL = default_URL + occs[occname[h]] #직군 자체의 value값으로 주소 할당 (0으로 할시 나오지 않음)
        else:
            URL = default_URL + jobvalue[jobnames[k]] #직군의 value값으로 설정
        
        wd.get(URL) #직군값이 들어가있는 해당 URL을 get
        time.sleep(3) #과부화 방지
        html = wd.page_source
        wanted = BeautifulSoup(html, 'html.parser')
        #연봉값을 가져오고 싶은데, 연봉 값이 페이지 안에 저장이 되어있지 않음
        #대신에 그래프 높이는 저장되어 있으므로, 그래프 폼의 y축 최댓값과 각 그래프의 높이를 비교하여 연봉값을 추출
        try:
            fullheight = wanted.select('defs > clipPath > rect') #그래프 폼 부분
            full_height = float(fullheight[0].get('height'))  #문자열로 되어있는 그래프 폼의 높이 값을 계산을 위해 float로 추출
        except: #페이지가 로드가 잘못되었을 경우
            wd.get(URL) #다시 전의 URL을 지정, 연결
            time.sleep(3) #과부화 방지
            html = wd.page_source
            wanted = BeautifulSoup(html, 'html.parser')
            fullheight = wanted.select('defs > clipPath > rect') #위의 코드와 동일
            full_height = float(fullheight[0].get('height'))

        heighttotal = wanted.select('g > g > g > g > path') #각 그래프의 모든 높이값이 저장되어 있음 
        heights = [] #높이를 저장할 리스트 생성
        for l in range(11): #신입~10년차 까지 11개의 값(신입, 1~10년차)만큼 반복
            heights.append(float(heighttotal[l].get('height'))) #각 그래프의 높이값을 heights 리스트에 저장

        yheight = wanted.select('g > text > tspan') #그래프 폼의 y축(연봉) 최댓값을 추출
        yheight = int(yheight[-1].string.replace(',','')) #8,000 이런식으로 저장되어 있으므로 콤마제거 후 정수로 받음

        yearmoney = [] #신입~10년차까지의 연봉을 저장할 리스트 초기화

        for j in range(len(heights)): #신입~10년차까지의 높이 수 만큼 반복(11번)
            yearmoney.append(int((heights[j] * yheight)/full_height)) #yearmoney 리스트에 그래프의 높이와 최대값을 이용한 연봉 계산값을 소수점을 버려 int형으로 저장

        #totaldf라는 연봉 데이터 프레임에 dfnum 인덱스값으로 저장
        totaldf.loc[dfnum] = [occname[h], jobnames[k], yearmoney[0], yearmoney[1], yearmoney[2], yearmoney[3], yearmoney[4], yearmoney[5], yearmoney[6], yearmoney[7], yearmoney[8], yearmoney[9], yearmoney[10]]
        dfnum = dfnum + 1 #index값 증가



totaldf

    

,직군,직무,신입,1년차,2년차,3년차,4년차,5년차,6년차,7년차,8년차,9년차,10년차
0,개발,전체,3536,3694,3982,4317,4624,4819,5177,5499,5859,6185,7057
1,개발,데이터 엔지니어,3613,3772,4138,4558,4985,5041,5462,6054,6427,6514,7699
2,개발,임베디드 개발자,3516,3674,3831,4136,4651,4809,5120,5556,5713,6176,6917
3,개발,자바 개발자,3353,3599,3839,4203,4434,4747,5083,5226,5639,6015,6859
4,개발,.NET 개발자,2974,3287,3545,4004,4297,4418,4919,5123,5357,5591,6507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,의료·제약·바이오,한의사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
269,의료·제약·바이오,방사선사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
270,의료·제약·바이오,병원 코디네이터,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
271,의료·제약·바이오,의무기록사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439


In [16]:
totaldf.to_csv('totaldf.csv')

In [14]:
totaldf

,직군,직무,신입,1년차,2년차,3년차,4년차,5년차,6년차,7년차,8년차,9년차,10년차
0,개발,전체,3536,3694,3982,4317,4624,4819,5177,5499,5859,6185,7057
1,개발,데이터 엔지니어,3613,3772,4138,4558,4985,5041,5462,6054,6427,6514,7699
2,개발,임베디드 개발자,3516,3674,3831,4136,4651,4809,5120,5556,5713,6176,6917
3,개발,자바 개발자,3353,3599,3839,4203,4434,4747,5083,5226,5639,6015,6859
4,개발,.NET 개발자,2974,3287,3545,4004,4297,4418,4919,5123,5357,5591,6507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,의료·제약·바이오,한의사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
227,의료·제약·바이오,방사선사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
228,의료·제약·바이오,병원 코디네이터,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
229,의료·제약·바이오,의무기록사,3247,3359,3614,3869,4046,4208,4306,4403,4953,5196,5439
